# Class 1 - Phase identification - Single-Bus structure

## Agenda
- Problem Description
- Problem Implementation
- Alternative Problems
- Wrap-Up

## Problem Description

- While the connectivity model of low voltage (LV) grids is usually known, the phase connectivity information of single-phase connected customers is often erroneous or missing. This is due to many reasons including maintenance and other load balancing initiatives that usually do not update phase connectivity information and do not keep track of its changes in a systematic way.

![distribution-graphic_v7-1000-x-750.jpeg](attachment:distribution-graphic_v7-1000-x-750.jpeg)
https://www.fortisalberta.com/about-us/our-company/blog/fortisalbertablog/2019/08/20/we-explain-the-complex-sophisticated-system-that-brings-electricity-to-you




- With the deployment of smart metering and the consequent availability
of high-resolution consumption data, phase connectivity
should be possible to estimate if data on per-phase aggregate energy
measurements would be available at substation sites with the
same resolutio.

![Problem1.png](attachment:Problem1.png)

<b>Consider <i>N</i> customers whose connection phase assumes one out of three possible labels <i>a, b, </i> or <i> c</i> and estimate the correct customer-to-label assignment based on <i>M</i> readings whose per-phase values are a <i>function</i> of the corresponding phase-connected individual customer reading.</b>

## Problem Description

![DASG_Problem1.PNG](attachment:DASG_Problem1.PNG)

Import Python Libraries

In [ ]:
import pandas as pd
import numpy as np
from numpy.random import randint   # To random values in the phases
from numpy.random import random   # To random values in the phases
import matplotlib.pyplot as plt


Parameters (It is possible to change to test different input data)

In [ ]:
nc=4                        # Number of consumers (1 to nc)                  %%Data Notes: nc=4
ts=60                       #start period of analysis (Can be from 1 to 96)  %%Data Notes: ts=60
te=71                       #Last period of analysis (Can be from 1 to 96)   %%Data Notes: te=71
phase =np.array([3,2,1,3])           #To obtain the same values of lecture notes
noise = 0
#phase = randint(1, 4, nc)  #To obtain random values

print ("The distribution of consumers in each phase is: ", phase)

The distribution of consumers in each phase is:  [3 2 1 3]


Import data (From Excel file)

In [ ]:
raw_data = np.array(pd.read_excel ('/Users/josegouveiasaramago/Programação/ADRI/Aula 1/Prob1_Conso_Data.xlsx', header=None))
print(raw_data.shape)

FileNotFoundError: ignored

Clean and organize the data (delete zeros and organize by consumers)

In [ ]:
checks=0
nr=1
data=np.zeros((1,96))
#h=np.arange(1/96, 1, 1/96).tolist()
h=raw_data[0:96,0]
for i in range(1,raw_data.shape[0]+1):
    if i==0:
        print(i)
    if raw_data[i-1,0]==h[checks]:
        checks=checks+1
    else:
        checks=0
    if checks==96:
        if np.sum(raw_data[i-96:i,1])!=0:
            data[nr-1,0:96]=raw_data[i-96:i,1]
            data.resize((nr+1,96))
            nr=nr+1
        checks=0
data.resize((nr-1,96))

data.shape[0]      #Can be deleted
print ("The number of consumers is ", data.shape[0], " and the number of periods is ", data.shape[1])

Select data from consumers and period (Truncate the original matrix) 

In [ ]:
data_Aux1=data[0:nc,:]
pw=data_Aux1[:,ts-1:te]

print ("The matrix 'pw' represents the power measured by the smart meter in each consumer (i) in each period (k)")
print ("In the lecture notes, this value is represented by X.")
print ("The value of X is:\n",np.transpose(4*pw))   # We should multiply by 4 to obtain the same values of the lectures. 
                                                    # In fact the original values are the average energy consumption for
                                                    # 15 minutes. To obtain the power, we should multiply by 4  

Consumers aggregation by phase and noise inclusion (normal distribution).

In [ ]:
X=np.transpose(4*pw)
y_perfect=np.zeros((X.shape[0],3))
#print(y_perfect.shape)

# In order to automate the process in case a different phase is assigned to these customers, we have chosen to make for cycles that will allow this function to work independently of each customer's phases
for j in range(X.shape[0]):
    for i in range(4):
        if phase[i]==1:
           y_perfect[j,0]=y_perfect[j,0]+X[j,i]
        elif phase[i]==2:
           y_perfect[j,1]=y_perfect[j,1]+X[j,i]
        elif phase[i]==3:
           y_perfect[j,2]=y_perfect[j,2]+X[j,i]
#print(y_perfect)
y_noise=np.zeros((y_perfect.shape[0],3))
sigma=0.5 #standart deviation

# Adding noise to the signal (normal distribution)
for i in range(y_perfect.shape[0]):
  for j in range(y_perfect.shape[1]):
    for value in np.random.normal(y_perfect[i][j], sigma, 20):
      y_noise[i][j]=value

#print(y_noise)


Multivariate Regression 

![B_equation.PNG](attachment:B_equation.PNG)

In [ ]:
# Calculating Beta for the perfect set
# Calculating the (X'*X)^-1
X_t = np.transpose(X)
temp = np.linalg.inv(np.matmul(X_t, X))

# Calculating ( (X'X)^-1 ) X' * Y
Beta_perfect = np.matmul(np.matmul(temp,  X_t), y_perfect)
 
# Calculating the estimative of Y using the Beta coefficients (verification)
Yest_perfect = np.matmul(X, Beta_perfect)
#print(Yest_perfect, Beta_perfect)

# Calculating Beta for the set with noise
# Calculating the (X'*X)^-1
X_t = np.transpose(X)
temp = np.linalg.inv(np.matmul(X_t, X))

# Calculating ( (X'X)^-1 ) X' * Y
Beta_noise = np.matmul(np.matmul(temp,  X_t), y_noise)
 
# Calculating the estimative of Y using the Beta coefficients (verification)
Yest_noise = np.matmul(X, Beta_noise)
#print(Yest_noise, Beta_noise)

#For used to confirm the phase of each client and to print the result
for i in range(nc):
  perfect_client=np.argmax(Beta_perfect[i], axis = None, out = None)
  noise_client=np.argmax(Beta_noise[i], axis = None, out = None)
  print('Client', i+1, 'is connected to the phase', perfect_client+1, 'and with noise to the phase', noise_client+1)

As one would expect, the beta of the signal without noise gives a matrix with a value of 1 in the phase the client is connected to, while in the other phases it gives approximately 0.

Analysing the signal with noise, we can conclude that the phase to which the customer is connected is the same as the one to which it is connected in the signal without noise, but their values are closer. 
The phase to which the customer is connected is the one with the highest value.

Plot

In [ ]:
x=np.ones((1,12))
for i in range(12):
  x[0][i]=(i+1)*x[0][i]

#Plot without noise
y_array=y_perfect[0:12,0]
y_array= np.reshape(y_array,(12,1))
plt.step(np.transpose(x), y_array, label='A')
y_array=y_perfect[0:12,1]
y_array= np.reshape(y_array,(12,1))
plt.step(np.transpose(x), y_array, label='B')
y_array=y_perfect[0:12,2]
y_array= np.reshape(y_array,(12,1))
plt.step(np.transpose(x), y_array, label='C')
plt.legend(title='Customer:')
plt.xlabel('time stamp[15 min]')
plt.ylabel('power[KW]')
plt.title('Per-phase plot without noise')
plt.show()

#Plot with noise
y_array=y_noise[0:12,0]
y_array= np.reshape(y_array,(12,1))
plt.step(np.transpose(x), y_array, label='A')
y_array=y_noise[0:12,1]
y_array= np.reshape(y_array,(12,1))
plt.step(np.transpose(x), y_array, label='B')
y_array=y_noise[0:12,2]
y_array= np.reshape(y_array,(12,1))
plt.step(np.transpose(x), y_array, label='C')
plt.legend(title='Customer:')
plt.xlabel('time stamp[15 min]')
plt.ylabel('power[KW]')
plt.title('Per-phase plot with noise')
plt.show()

#Plot of Consumers Readings
y_array=X[0:12,0]
y_array= np.reshape(y_array,(12,1))
plt.step(np.transpose(x), y_array, label='1')
y_array=X[0:12,1]
y_array= np.reshape(y_array,(12,1))
plt.step(np.transpose(x), y_array, label='2')
y_array=X[0:12,2]
y_array= np.reshape(y_array,(12,1))
plt.step(np.transpose(x), y_array, label='3')
y_array=X[0:12,3]
y_array= np.reshape(y_array,(12,1))
plt.step(np.transpose(x), y_array, label='4')
plt.legend(title='Customer:')
plt.xlabel('time stamp[15 min]')
plt.ylabel('power[KW]')
plt.title('Customer Reading')
plt.show()


# Extra Challenges
Think about an interesting variation to this problem or a different way to solve and implement it !!!

Some ideas:
- What happens if we have two consumers with the same consumption ? What about if the difference is very small ? Can we quantify the sensitivity ?
- What happens if we have three-phase clients ? Can we follow the same approach ?

In [ ]:
#Se os clientes tivessem os mesmos consumos ou consumos muito semelhantes teríamos gráficos sobrepostos o que 
#dificultaria a leitura dos dados.
#A sensibilidade corresponde à unidade da menor escala do gráfico, quanto maior a escala menor a sensibilidade 
#(sensibilidade inversamente proporcional à escala do gráfico)
print('If the customers had the same or very similar consumptions we would have overlapping graphs which would make it difficult to read the data. The sensitivity corresponds to the unit of the smallest scale of the graphic, the larger the scale the lower the sensitivity (sensitivity inversely proportional to the scale of the graph)')


print('')

#Num modelo trifasico, se não houver ruido podemos ter a mesma approach, visto que o valor de cada Beta corresponde à percentagem de potencia
#que estamos a tirar de cada fase. No entanto, se houver ruído (ou erros) os betas não vão ser coerentes nessa percentagem, logo não poderiamos
#abordar da mesma forma
print('If we had three-phase clients we would have to have a matrix of matrixes. In this matrix would be the consumptions of each client per phase. We would have to generate the matrix of each client and then generate the matrix of the phases where the matrixes of each client would be inserted.')